# Example notebook for the functions contained in CRYSTALpytools.crystal_io

### Crystal_input class

In [1]:
from CRYSTALpytools.crystal_io import Crystal_input

#### Create a crystal input object by directly calling methods

In [2]:
mgo_input = Crystal_input()
mgo_input.geom.title('MGO BULK - GEOMETRY TEST')
mgo_input.geom.crystal(225, # Space group
                      [4.217], # Minimal set of lattice parameters
                      [[12, 0., 0., 0.], # Atomic labels and coordinates
                       [8, 0.5, 0.5, 0.5]]
                      )
mgo_input.basisset.basisset('POB-DZVP')
mgo_input.scf.dft.xcfunc('B3LYP')
mgo_input.scf.dft.xxlgrid()
mgo_input.scf.tolinteg(7, 7, 7, 7, 14)
mgo_input.scf.shrink(12, 24)
mgo_input.scf.maxcycle(70)
mgo_input.scf.fmixing(70)
mgo_input.scf.diis()

print(mgo_input.data)

MGO BULK - GEOMETRY TEST
CRYSTAL
0   0   0   
225 
4.217000     
2 
12     0.00000000   0.00000000   0.00000000 
8      0.50000000   0.50000000   0.50000000 
ENDGEOM
BASISSET
POB-DZVP 
DFT
XXLGRID
B3LYP 
ENDDFT
DIIS
FMIXING
70 
TOLINTEG
7 7 7 7 14 
SHRINK
12 24 
ENDSCF



#### Create a crystal input object from blocks

In [3]:
geom_block = \
"""MGO BULK - GEOMETRY TEST
CRYSTAL
0 0 0
225
4.217
2
12 0.    0.    0.
8 0.5   0.5   0.5
END
"""

bs_block   = \
"""BASISSET
POB-DZVP
"""
func_block = \
"""DFT
B3LYP
XXLGRID
ENDDFT
"""
scf_block  = \
"""TOLINTEG
7 7 7 7 14
SHRINK
12 24
MAXCYCLE
200
FMIXING
70
DIIS
ENDSCF
"""
mgo_input = Crystal_input()
mgo_input.set_geom(geom_block)
mgo_input.set_basisset(bs_block)
mgo_input.scf.set_dft(func_block) # DFT is a sub-block of SCF block
mgo_input.set_scf(scf_block)
print(mgo_input.data)

MGO BULK - GEOMETRY TEST
CRYSTAL
0 0 0
225
4.217
2
12 0.    0.    0.
8 0.5   0.5   0.5
ENDGEOM
BASISSET
POB-DZVP
ENDBS
DIIS
MAXCYCLE
200
FMIXING
70
TOLINTEG
7 7 7 7 14
SHRINK
12 24
ENDSCF



#### Create a crystal input object from an existing input file

The existing file can act as a template. By calling the corresponding method one can change, delete and substitute keywords.

In [4]:
mgo_input = Crystal_input().from_file('data/mgo.d12')
# Set Optgeom block
mgo_input.geom.optgeom.toldex(0.0012)
mgo_input.geom.optgeom.toldeg(0.0003)
print(mgo_input.geom.data)
# Clean TOLDEG command
mgo_input.geom.optgeom.toldeg('')
print(mgo_input.geom.optgeom.data)
# Write the updated object to file
mgo_input.to_file('data/mgo_inputtest.d12')

MGO BULK - GEOMETRY TEST
CRYSTAL
0 0 0
225
4.217
2
12 0.    0.    0.
8 0.5   0.5   0.5
OPTGEOM
TOLDEG
0.0003 
TOLDEX
0.0012 
ENDOPT
ENDGEOM

OPTGEOM
TOLDEX
0.0012 
ENDOPT



#### Set geometry

The user can set the geometry input by a CIF file or a pymatgen structure when needed. There are 2 available options:

1. Use the 'EXTERNAL' keyword and `Crystal_input` object automatically convert the geometry entry to a gui file with default settings (recommended)  
2. Use the 'CRYSTAL' keyword. Limited to 3D structures.

In [5]:
paracetamol = Crystal_input()
paracetamol.geom_from_cif('data/paracetamol.cif', keyword='CRYSTAL')
print(paracetamol.geom.data)
paracetamol = Crystal_input().geom_from_cif('data/paracetamol.cif',
                                            gui_name='data/paracetamol.gui')

Generated by CRYSTALpytools
CRYSTAL
0   0   0   
14  
7.073000     9.166000     12.667000    115.510000   
20 
1      0.37160000   0.92980000   0.49880000 
1      0.24220000   0.76340000   0.33040000 
1      0.77660000   0.60000000   0.43090000 
1      0.90770000   0.76640000   0.60050000 
1      0.79990000   0.53530000   0.25760000 
1      0.10820000   0.95930000   0.69510000 
1      0.56840000   0.08530000   0.89610000 
1      0.29390000   0.07980000   0.85010000 
1      0.40860000   0.21910000   0.80230000 
6      0.14960000   0.85790000   0.56187000 
6      0.24320000   0.85758000   0.48526000 
6      0.17040000   0.76206000   0.39004000 
6      0.00430000   0.66838000   0.37078000 
6      0.90790000   0.67092000   0.44601000 
6      0.98060000   0.76560000   0.54106000 
6      0.39830000   0.01599000   0.71975000 
6      0.41680000   0.10369000   0.82440000 
7      0.21229000   0.94984000   0.66089000 
8      0.94130000   0.57511000   0.27811000 
8      0.54450000   0.00741000   0

#### Set basis set

The user can set basis set by string (as shown before), a text file, downloading it from [Basis Set Exchange (BSE)](https://www.basissetexchange.org/) or a `CRYSTALpytools.base.basisset.BasisSetBASE` object. CRYSTALpytools calls BSE's API to convert and format basis set files, so file/string in other formats (Gaussian, for example) can also be used.

Effective core pesudopotentials are supported except the free-format definitions ('INPUT' and 'INPSOC').

In [6]:
# Downlaod basis set from BSE
mgo_input.basisset.from_bse('6-311G*', ['Mg', 'O'])
print(mgo_input.basisset.data)

8    5    
0 0 6  0.00  1.00  
   8.5885000000E+03     1.8951500000E-03
   1.2972300000E+03     1.4385900000E-02
   2.9929600000E+02     7.0732000000E-02
   8.7377100000E+01     2.4000100000E-01
   2.5678900000E+01     5.9479700000E-01
   3.7400400000E+00     2.8080200000E-01
0 1 3  0.00  1.00  
   4.2117500000E+01     1.1388900000E-01     3.6511400000E-02
   9.6283700000E+00     9.2081100000E-01     2.3715300000E-01
   2.8533200000E+00    -3.2744700000E-03     8.1970200000E-01
0 1 1  0.00  1.00  
   9.0566100000E-01     1.0000000000E+00     1.0000000000E+00
0 1 1  0.00  1.00  
   2.5561100000E-01     1.0000000000E+00     1.0000000000E+00
0 3 1  0.00  1.00  
   1.2920000000E+00     1.0000000000E+00
12   12   
0 0 6  0.00  1.00  
   4.3866500000E+04     9.1800000000E-04
   6.6053700000E+03     7.0470000000E-03
   1.5132600000E+03     3.5941000000E-02
   4.3231700000E+02     1.4146100000E-01
   1.4214900000E+02     4.2676400000E-01
   5.1398300000E+01     4.9797500000E-01
0 0 3  0.00  1.

/home/e89/e89/hyz20ic/app/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/CRYSTALpytools/base/crysd12.py:880: UserWarning: The previous basis set will be erased.
  self._check_bs(append)


In [7]:
# Read a Gaussian STO-3G basis set
mgo_input.basisset.from_file('data/mgo_inputbs.gbs', 'gaussian94')
print(mgo_input.basisset.data)

8    2    
0 0 3  0.00  1.00  
   1.3070932140E+02     1.5432896730E-01
   2.3808866050E+01     5.3532814230E-01
   6.4436083130E+00     4.4463454220E-01
0 1 3  0.00  1.00  
   5.0331513190E+00    -9.9967229190E-02     1.5591627500E-01
   1.1695961250E+00     3.9951282610E-01     6.0768371860E-01
   3.8038896000E-01     7.0011546890E-01     3.9195739310E-01
12   3    
0 0 3  0.00  1.00  
   2.9923741370E+02     1.5432896730E-01
   5.4506468450E+01     5.3532814230E-01
   1.4751577520E+01     4.4463454220E-01
0 1 3  0.00  1.00  
   1.5121823520E+01    -9.9967229190E-02     1.5591627500E-01
   3.5139865790E+00     3.9951282610E-01     6.0768371860E-01
   1.1428574980E+00     7.0011546890E-01     3.9195739310E-01
0 1 3  0.00  1.00  
   1.3954482930E+00    -2.1962036900E-01     1.0587604290E-02
   3.8932653180E-01     2.2559543360E-01     5.9516700530E-01
   1.5237976590E-01     9.0039842600E-01     4.6200101200E-01
99   0
ENDBS



/home/e89/e89/hyz20ic/app/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/CRYSTALpytools/base/crysd12.py:928: UserWarning: The previous basis set will be erased.
  self._check_bs(append)


#### Set conventional atomic numbers

It is strongly recommended to set conventional atomic numbers by calling methods such as `Crystal_input.geom.crystal()` and manually type corresponding atomic numbers. Even though `zconv` entry can be provided when using `Crystal_input.geom_from_cif` and `Crystal_input.geom_from_pmg`, sequences of atoms in the input file might not be consistent with sequences in output.

To set conventional atomic numbers for basis sets, the user can either explicitly define them in string / file, or call BSE twice:

In [8]:
obj = Crystal_input()
obj.basisset.from_bse('6-311G*', ['C', 'H']) # Normal definition
obj.basisset.from_bse('def2-TZVP', [['H', 101]], append=True) # Define a second basis set for H with label 101
print(obj.basisset.data)

1    3    
0 0 3  0.00  1.00  
   3.3865000000E+01     2.5493800000E-02
   5.0947900000E+00     1.9037300000E-01
   1.1587900000E+00     8.5216100000E-01
0 0 1  0.00  1.00  
   3.2584000000E-01     1.0000000000E+00
0 0 1  0.00  1.00  
   1.0274100000E-01     1.0000000000E+00
6    5    
0 0 6  0.00  1.00  
   4.5632400000E+03     1.9666500000E-03
   6.8202400000E+02     1.5230600000E-02
   1.5497300000E+02     7.6126900000E-02
   4.4455300000E+01     2.6080100000E-01
   1.3029000000E+01     6.1646200000E-01
   1.8277300000E+00     2.2100600000E-01
0 1 3  0.00  1.00  
   2.0964200000E+01     1.1466000000E-01     4.0248700000E-02
   4.8033100000E+00     9.1999900000E-01     2.3759400000E-01
   1.4593300000E+00    -3.0306800000E-03     8.1585400000E-01
0 1 1  0.00  1.00  
   4.8345600000E-01     1.0000000000E+00     1.0000000000E+00
0 1 1  0.00  1.00  
   1.4558500000E-01     1.0000000000E+00     1.0000000000E+00
0 3 1  0.00  1.00  
   6.2600000000E-01     1.0000000000E+00
101  4    
0 0 3

### Crystal_output class

In [9]:
from CRYSTALpytools.crystal_io import Crystal_output

#### Read output case 1: Periodic system

In [10]:
mgo_output = Crystal_output().read_cry_output('data/mgo_optgeom.out')
mgo_output

- Substracting basice energy and electronic information

In [11]:
# Initial SCF energy
print("Final energy = %s eV \n" % mgo_output.get_final_energy())
# Initial SCF energy, an alternative way
mgo_output.get_convergence(history=False)
print("Final energy = %s eV \n" % mgo_output.final_energy)

# Initial SCF Fermi energy
print("Fermi energy = %s eV \n" % mgo_output.get_fermi_energy())

# Initial SCF Band gap
print("Band gap = %s eV \n" % mgo_output.get_band_gap())

# Mulliken charge, Note that is based on the optimized geometry
print("Mulliken charge (e) = ", mgo_output.get_mulliken_charges())

Final energy = -7495.340914150903 eV 

Final energy = -7495.340914150903 eV 

Fermi energy = -4.136710311917215 eV 

Band gap = 7.1237 eV 

Mulliken charge (e) =  [10.129  9.871]


- Substracting SCF convergence

In [12]:
mgo_output.get_scf_convergence()
# Convergence status
print('SCF convergence status = {}\n'.format(mgo_output.scf_status))
print('SCF number of cycles = {:d}\n'.format(mgo_output.scf_cycles))

# Energy iteration history
print('Total energy (eV) = \n', mgo_output.scf_energy, '\n')
print('Energy difference (eV)= \n', mgo_output.scf_deltae, '\n')

# Fermi level iteration history
print('Fermi level (eV) = \n', mgo_output.get_fermi_energy(history=True), '\n')

# Band gap iteration history
print('Band gap (eV) = \n', mgo_output.get_band_gap(history=True), '\n')

SCF convergence status = converged

SCF number of cycles = 8

Total energy (eV) = 
 [-7476.82733948 -7489.97945008 -7494.91919331 -7495.33640687
 -7495.33787794 -7495.33800249 -7495.33800425 -7495.33800435] 

Energy difference (eV)= 
 [-7.48313122e+03 -1.31430996e+01 -4.95247230e+00 -4.16334210e-01
 -1.47213600e-03 -1.24628149e-04 -1.75513441e-06 -1.02586926e-07] 

Fermi level (eV) = 
 [-8.24883759 -4.59365942 -4.10322534 -4.12058103 -4.14111121 -4.13676256
 -4.13671031] 

Band gap (eV) = 
 [8.4379 6.6125 7.0523 7.1034 7.1228 7.1235 7.1237] 



- Substracting forces

Note that the forces computed at the initial SCF step is read. To read the final forces, set `initial=False` and use 'ONELOG' keyword in CRYSTAL input file. `grad=True` is valid for optimizations only.

In [13]:
mgo_output.get_forces(initial=True, grad=True)

print('Unit: Hartree/Bohr\n')
print("Forces on cell = \n %s \n\n Forces on atoms = \n %s \n\n RMS Gradient = \n %s \n\n" %
      (mgo_output.forces_cell, mgo_output.forces_atoms, mgo_output.opt_rmsgrad))

Unit: Hartree/Bohr

Forces on cell = 
 [[-0.00148273  0.00148273  0.00148273]
 [ 0.00148273 -0.00148273  0.00148273]
 [ 0.00148273  0.00148273 -0.00148273]] 

 Forces on atoms = 
 [[-4.18889278e-16 -4.11372640e-16 -4.14981546e-16]
 [ 4.14263349e-16  4.13107364e-16  4.14981546e-16]] 

 RMS Gradient = 
 [2.0466e-02 4.6930e-03 6.9000e-05 3.2000e-05] 




- Substracting basic geometry information

Option 1: From `get_*` methods

By setting `initial=False` the optimized geometry can be printed. In other cases such as SCF calculations, `initial=False` has no effect but takes slightly longer to get results

In [14]:
# Symmetry operators
print("Symmetry operators = \n %s \n" % mgo_output.get_symmops())

# Pre-optimized geometry
print('Pre-optimized geometry\n', mgo_output.get_geometry(initial=True), '\n')

# Optimized geometry
print('Optimized geometry\n', mgo_output.get_geometry(initial=False), '\n')
# Optimized geometry, an alternative, which returns a list of key informations
print('Using get_last_geom()\n', mgo_output.get_last_geom(write_gui_file=False), '\n')

# Lattice matrix
print('Lattice matrix:\n', mgo_output.get_lattice(initial=True), '\n')

# Reprocal lattice matrix
print('Reprocal lattice matrix\n', mgo_output.get_reciprocal_lattice(initial=True), '\n')

# Cell expansion matrix
print('Cell expansion matrix\n', mgo_output.get_trans_matrix(), '\n')

# Primitive optimized geometry (If supercell expansion is applied)
print('Primitive optimized geometry\n', mgo_output.get_primitive_geometry(initial=False), '\n')

# Primitive lattice matrix (If supercell expansion is applied)
print('Primitive lattice matrix\n', mgo_output.get_primitive_lattice(initial=True), '\n')

# Primitive reciprocal lattice matrix (If supercell expansion is applied)
print('Primitive reciprocal lattice matrix\n', mgo_output.get_primitive_reciprocal_lattice(initial=True), '\n')

Symmetry operators = 
 [[[ 1.  0.  0.]
  [ 0.  1.  0.]
  [ 0.  0.  1.]
  [ 0.  0.  0.]]

 [[ 0.  1.  0.]
  [ 1.  0.  0.]
  [-1. -1. -1.]
  [ 0.  0.  0.]]

 [[-1. -1. -1.]
  [ 0.  0.  1.]
  [ 0.  1.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  1.]
  [-1. -1. -1.]
  [ 1.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  1.]
  [ 1.  0.  0.]
  [ 0.  1.  0.]
  [ 0.  0.  0.]]

 [[ 0.  1.  0.]
  [ 0.  0.  1.]
  [ 1.  0.  0.]
  [ 0.  0.  0.]]

 [[ 1.  0.  0.]
  [ 0.  0.  1.]
  [-1. -1. -1.]
  [ 0.  0.  0.]]

 [[ 1.  0.  0.]
  [-1. -1. -1.]
  [ 0.  1.  0.]
  [ 0.  0.  0.]]

 [[-1. -1. -1.]
  [ 0.  1.  0.]
  [ 1.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  1.]
  [ 0.  1.  0.]
  [-1. -1. -1.]
  [ 0.  0.  0.]]

 [[ 0.  1.  0.]
  [-1. -1. -1.]
  [ 0.  0.  1.]
  [ 0.  0.  0.]]

 [[-1. -1. -1.]
  [ 1.  0.  0.]
  [ 0.  0.  1.]
  [ 0.  0.  0.]]

 [[ 0. -1.  0.]
  [-1.  0.  0.]
  [ 0.  0. -1.]
  [ 0.  0.  0.]]

 [[-1.  0.  0.]
  [ 0. -1.  0.]
  [ 1.  1.  1.]
  [ 0.  0.  0.]]

 [[ 0.  0. -1.]
  [ 1.  1.  1.]
  [ 0. -1.  0.]
  [ 0

Option 2: Basic properties can be directly called as properties

For atomic coordinates, that is equivalent to `get_geometry(initial=True)`, i.e., the initial geometry.

In [15]:
# Space group number and symbol
print('Space group = {:d} {:s}\n'.format(mgo_output.sg_number, mgo_output.sg_symbol))
# Number of atoms
print('Number of atoms = {:d}\n'.format(mgo_output.n_atoms))
# Atomic symbols
print('Species = ', mgo_output.atom_symbols, '\n')
# Conventional atomic numbers, consistent with CRYSTAL (e.g., 101 for H with pesudopotentials)
print('Conventional atomic numbers = ', mgo_output.atom_numbers, '\n')
# Atomic positions, consistent with CRYSTAL (e.g., Fractional, Fractional, Cartesian for 2D periodic systems)
print('Atomic positions = \n', mgo_output.atom_positions, '\n')
# Fractional atomic positions
print('Fractional atomic positions = \n', mgo_output.atom_positions_frac, '\n')
# Cartesian atomic positions
print('Cartesian atomic positions = \n', mgo_output.atom_positions_cart, '\n')

Space group = 225 F M 3 M

Number of atoms = 2

Species =  ['Mg', 'O'] 

Conventional atomic numbers =  [12  8] 

Atomic positions = 
 [[0.  0.  0. ]
 [0.5 0.5 0.5]] 

Fractional atomic positions = 
 [[0.  0.  0. ]
 [0.5 0.5 0.5]] 

Cartesian atomic positions = 
 [[0.     0.     0.    ]
 [2.1085 2.1085 2.1085]] 



- Substracting Optimization convergence and trajectory

Trajectory is written in 'data/mgo_optgeom.opttraj' in CRYSTAL gui format.

In [17]:
!mkdir data/mgo_optgeom.opttraj
mgo_output.get_opt_convergence(write_gui=True, gui_name='data/mgo_optgeom.opttraj/mgo')

print('Optimization convergence status = {:s}\n'.format(mgo_output.opt_status))
print('Optimization cycles = {:d}\n'.format(mgo_output.opt_cycles))
print('Optimization energy history (eV) = \n', mgo_output.opt_energy, '\n')
print('Optimization energy difference (eV) = \n', mgo_output.opt_deltae, '\n')
print('Optimization max gradient (Hartree/Bohr) = \n', mgo_output.opt_maxgrad, '\n')
print('Optimization RMS gradient (Hartree/Bohr) = \n', mgo_output.opt_rmsgrad, '\n')
print('Optimization max displacement (Bohr) = \n', mgo_output.opt_maxdisp, '\n')
print('Optimization RMS displacement (Bohr) = \n', mgo_output.opt_rmsdisp, '\n')

Optimization convergence status = converged

Optimization cycles = 4

Optimization energy history (eV) = 
 [ -275.44859114 -7495.3406783  -7495.34087403 -7495.34091415] 

Optimization energy difference (eV) = 
 [-2.75448591e+02 -2.67406293e-03 -1.95731501e-04 -4.01367947e-05] 

Optimization max gradient (Hartree/Bohr) = 
 [2.0466e-02 4.6930e-03 6.9000e-05 3.2000e-05] 

Optimization RMS gradient (Hartree/Bohr) = 
 [2.0466e-02 4.6930e-03 6.9000e-05 3.2000e-05] 

Optimization max displacement (Bohr) = 
 [0.000e+00 2.177e-03 3.200e-05 1.500e-05] 

Optimization RMS displacement (Bohr) = 
 [0.000e+00 2.177e-03 3.200e-05 1.500e-05] 



#### 0D system

In [18]:
co_output = Crystal_output().read_cry_output('data/co.out')

In [19]:
# Number of atoms
print('Number of atoms = {:d}\n'.format(co_output.n_atoms))
# Atomic symbols
print('Species = ', co_output.atom_symbols, '\n')
# Atomic positions, consistent with CRYSTAL (e.g., Fractional, Fractional, Cartesian for 2D periodic systems)
print('Atomic positions = \n', co_output.atom_positions, '\n')

# Final energy
print("Final energy = %s eV \n" % co_output.get_final_energy())

# Fermi energy
print("Fermi energy = %s eV \n" % co_output.get_fermi_energy())

# Primitive lattice
print("Primitive lattice \n %s \n" % co_output.get_primitive_lattice())

# Reciprocal lattice
print("Reciprocal lattice \n %s \n" % co_output.get_reciprocal_lattice())

# Band gap
print("Band gap = %s eV \n" % co_output.get_band_gap())

# Last geometry
print("Last geometry = \n %s \n" % co_output.get_last_geom())

# Symmetry operators
print("Symmetry operators = \n %s \n" % co_output.get_symmops())

# Forces
co_output.get_forces()
print("Forces on cell = \n %s \n\n Forces on atoms = \n %s \n\n" %
      (co_output.forces_cell, co_output.forces_atoms))

# Scf convergence
co_output.get_scf_convergence()
print("SCF energy history = \n %s \n\n Energy difference = \n %s \n\n" %
      (co_output.scf_energy, co_output.scf_deltae))

Number of atoms = 2

Species =  ['C', 'O'] 

Atomic positions = 
 [[0.  0.  0. ]
 [0.8 0.5 0.4]] 

Final energy = -3040.063479187389 eV 

Fermi energy = -7.247933954451301 eV 

Primitive lattice 
 None 

Reciprocal lattice 
 None 

Band gap = 10.9012 eV 

Last geometry = 
 [[[500.0, 0.0, 0.0], [0.0, 500.0, 0.0], [0.0, 0.0, 500.0]], array([6, 8]), [[0.0, 0.0, 0.0], [0.8, 0.5, 0.4]]] 

Symmetry operators = 
 [] 

Forces on cell = 
 [] 

 Forces on atoms = 
 [[-0.48652346 -0.30407721 -0.24326191]
 [ 0.48652346  0.30407721  0.24326191]] 


SCF energy history = 
 [-3050.41709057 -3037.8127751  -3037.86317042 -3037.87780122
 -3037.87917172 -3037.87917184 -3037.87917184] 

 Energy difference = 
 [-3.04767526e+03  1.25988718e+01 -5.03410646e-02 -1.46397258e-02
 -1.37145387e-03 -1.17281075e-07 -7.48313122e-10] 




/home/e89/e89/hyz20ic/app/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/CRYSTALpytools/crystal_io.py:716: UserWarning: 0D system. No lattice.
  warnings.warn('0D system. No lattice.')
/home/e89/e89/hyz20ic/app/anaconda3/envs/crystal_py3.9/lib/python3.9/site-packages/CRYSTALpytools/crystal_io.py:431: UserWarning: 0D system. No lattice.
  warnings.warn('0D system. No lattice.')


### Properties_input class

#### bands from k points coordinates

In [20]:
from CRYSTALpytools.crystal_io import Properties_input

#Create the bands input object
bands_input = Properties_input()
#Add the newk block to the input object
bands_input.make_newk_block(12,24)

#Prepare the band_block
k_path = [[0,0,0],[0.5,0,0],[0.5,0.5,0.5],[0.25,0,0.5]]
n_kpoints = 200
first_band = 1
last_band = 26
bands_input.make_bands_block(k_path,n_kpoints,first_band,last_band)

#Write the input
bands_input.write_properties_input('data/bands_input_1.d3')

In [21]:
bands_input.property_block

['BAND\n',
 'BAND STRUCTURE CALCULATION\n',
 '3 4 200 1 26 1 0\n',
 '0 0 0  2 0 0\n',
 '2 0 0  2 2 2\n',
 '2 2 2  1 0 2\n',
 'END\n']

#### bands from pymatgen HighSymmKpath object

In [23]:
from pymatgen.symmetry.bandstructure import HighSymmKpath
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

from CRYSTALpytools.crystal_io import Crystal_output
from CRYSTALpytools.crystal_io import Properties_input
from CRYSTALpytools.convert import cry_out2pmg


#Create the bands input object
bands_input = Properties_input()
#Add the newk block to the input object
bands_input.make_newk_block(12,24)

#Read the structure
mgo = cry_out2pmg('data/mgo.out')
mgo_prim = SpacegroupAnalyzer(mgo).get_primitive_standard_structure(international_monoclinic=False)

#Obtain the k path object
k_path = HighSymmKpath(mgo_prim)

n_kpoints = 200
first_band = 1
last_band = 26
bands_input.make_bands_block(k_path,n_kpoints,first_band,last_band)

#Write the input
bands_input.write_properties_input('data/bands_input_2.d3')

In [24]:
bands_input.property_block

['BAND\n',
 'BAND STRUCTURE CALCULATION\n',
 '11 8 200 1 26 1 0\n',
 '0 0 0  4 0 4\n',
 '4 0 4  4 2 5\n',
 '4 2 5  2 2 5\n',
 '2 2 5  0 0 0\n',
 '0 0 0  4 4 4\n',
 '4 4 4  5 2 5\n',
 '5 2 5  4 2 5\n',
 '4 2 5  4 4 4\n',
 '4 4 4  2 2 5\n',
 '2 2 5  5 2 5\n',
 '5 2 5  4 0 4\n',
 'END\n']

#### doss

In [25]:
from CRYSTALpytools.crystal_io import Properties_input

#Create the doss input object
doss_input = Properties_input()
#Add the newk block to the input object
doss_input.make_newk_block(12,24)

#Prepare the doss_block

doss_input.make_doss_block(n_points=200,e_range=[-5,15],plotting_option=2,poly=12,print_option=1)

#Write the input
doss_input.write_properties_input('data/doss_input.d3')

In [26]:
!cat data/doss_input.d3

NEWK
12 24
1 0
DOSS
0 200 -1 -1 2 12 1
-0.18374661087827499 0.551239832634825
END


#### pdoss (atoms)

In [27]:
from CRYSTALpytools.crystal_io import Properties_input

#Create the doss input object
pdoss_input = Properties_input()
#Add the newk block to the input object
pdoss_input.make_newk_block(12,24)

#Prepare the pdoss_block
projections = [[1],[2]]
pdoss_input.make_pdoss_block(projections,proj_type='atom',n_points=200,band_range=[1,26],
                       plotting_option=2,poly=12,print_option=1)

#Write the input
pdoss_input.write_properties_input('data/pdoss_input.d3')

In [28]:
!cat data/pdoss_input.d3

NEWK
12 24
1 0
DOSS
2 200 1 26 2 12 1
-1 1
-1 2
END


#### pdoss (ao)

In [29]:
from CRYSTALpytools.crystal_io import Properties_input

#Create the doss input object
pdoss_input = Properties_input()
#Add the newk block to the input object
pdoss_input.make_newk_block(12,24)

#Prepare the pdoss_block
projections = [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
              [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]]
pdoss_input.make_pdoss_block(projections,proj_type='ao',n_points=200,band_range=[1,26],
                       plotting_option=2,poly=12,print_option=1)

#Write the input
pdoss_input.write_properties_input('data/pdoss_input_ao.d3')

In [30]:
!cat data/pdoss_input_ao.d3

NEWK
12 24
1 0
DOSS
2 200 1 26 2 12 1
13 1 2 3 4 5 6 7 8 9 10 11 12 13
13 14 15 16 17 18 19 20 21 22 23 24 25 26
END


### Properties_output class

In [31]:
from CRYSTALpytools.crystal_io import Properties_output

#### Band structure
Either 'BAND.DAT' or 'fort.25' format can be used.

In [32]:
mgo_bands = Properties_output()
mgo_bands.read_electron_band('data/mgo_BAND_dat.BAND')
mgo_bands

#### Density of states (DOS)
Either 'DOSS.DAT' or 'fort.25' format can be used.

In [33]:
mgo_doss = Properties_output()
mgo_doss = mgo_doss.read_electron_dos('data/mgo_DOSS_dat.DOSS')
mgo_doss

### Crystal_gui class

In [34]:
from CRYSTALpytools.crystal_io import Crystal_gui

#### Read CRYSTAL GUI file
A gui file is read by `Crystal_gui().read_gui()` method. The file can be converted into other geometry data formats including Pymatgen Structure/Molecule object and CIF file using functions in `CRYSTALpytools.convert`.

In [36]:
gui = Crystal_gui().read_gui('data/mgo_optgeom.opttraj/mgo-opt004.gui')
print('Dimensionality = {:d}\n'.format(gui.dimensionality))
print('Space group = {:d}\n'.format(gui.space_group))
print('N atoms = {:d}\n'.format(gui.n_atoms))
print('N symmetry operations = {:d}\n'.format(gui.n_symmops))

Dimensionality = 3

Space group = 225

N atoms = 2

N symmetry operations = 48



#### Write data as CRYSTAL GUI file
Other geometry data formats including Pymatgen Structure/Molecule object and CIF can be read and converted into CRYSTAL gui file.

* 3D structure

A conventional MgO cell is generated here, which will not be reduced to ensure consistency. To get the irreducible representation, use `CRYSTALpytools.geometry.refine_geometry()` function

In [37]:
from CRYSTALpytools.convert import cry_pmg2gui
from pymatgen.core import Structure, Lattice

# A conventional cell is generated here, which will not be reduced to maintain consistency
substrate = Structure.from_spacegroup("Fm-3m", Lattice.cubic(4.217), ["Mg",'O'], [[0, 0, 0],[0.5,0.5,0.5]])
mgo_gui = cry_pmg2gui(substrate, gui_file=None)
mgo_gui.write_gui('data/mgo_write_gui.gui')

In [38]:
! cat data/mgo_write_gui.gui

   3   1   1
  4.217000000000E+00  0.000000000000E+00  0.000000000000E+00
  0.000000000000E+00  4.217000000000E+00  0.000000000000E+00
  0.000000000000E+00  0.000000000000E+00  4.217000000000E+00
   48
      1.000000000000      0.000000000000      0.000000000000
      0.000000000000      1.000000000000      0.000000000000
      0.000000000000      0.000000000000      1.000000000000
      0.000000000000      0.000000000000      0.000000000000
     -1.000000000000      0.000000000000      0.000000000000
      0.000000000000     -1.000000000000      0.000000000000
      0.000000000000      0.000000000000     -1.000000000000
      0.000000000000      0.000000000000      0.000000000000
      0.000000000000     -1.000000000000      0.000000000000
      1.000000000000      0.000000000000      0.000000000000
      0.000000000000      0.000000000000      1.000000000000
      0.000000000000      0.000000000000      0.000000000000
      0.000000000000      1.000000000000      0.000000000000
     

* 2D slab

A MgO (100) slab is generated. The thickness of vacuum layer (which is not very meaningful since CRYSTAL is based on atomic orbitals) is set to be 25 Angstrom, instead of the default of 500 Angstrom in z direction. 

In [39]:
from pymatgen.core import Structure, Lattice
from pymatgen.core.surface import SlabGenerator

#Generate bulk structure
substrate = Structure.from_spacegroup("Fm-3m", Lattice.cubic(4.217), ["Mg",'O'], [[0, 0, 0],[0.5,0.5,0.5]])
#Generate slab
substrate = SlabGenerator(substrate, (1,0,0), 5., 10., center_slab=True).get_slab()

# Pymatgen get_slab will return to a 3D structure. Use pbc to force the
# implementation of 2D periodicity otherwise the vaccum input is invalid
mgo_slab_gui = cry_pmg2gui(substrate, gui_file=None, pbc=[True, True, False], vacuum=25)
mgo_slab_gui.write_gui('data/mgo_100_write_gui.gui',mgo_slab_gui)

In [40]:
! cat data/mgo_100_write_gui.gui

   2   1   1
  2.981869296264E+00  0.000000000000E+00  0.000000000000E+00
  0.000000000000E+00  2.981869296264E+00  0.000000000000E+00
  0.000000000000E+00  0.000000000000E+00  3.132550000000E+01
   16
      1.000000000000      0.000000000000      0.000000000000
      0.000000000000      1.000000000000      0.000000000000
     -0.000000000000     -0.000000000000      1.000000000000
      0.000000000000      0.000000000000      0.000000000000
      0.000000000000     -1.000000000000      0.000000000000
      1.000000000000     -0.000000000000      0.000000000000
     -0.000000000000     -0.000000000000      1.000000000000
      0.000000000000      0.000000000000      0.000000000000
     -1.000000000000     -0.000000000000      0.000000000000
      0.000000000000     -1.000000000000      0.000000000000
     -0.000000000000     -0.000000000000      1.000000000000
      0.000000000000      0.000000000000      0.000000000000
     -0.000000000000      1.000000000000      0.000000000000
     